In [34]:
#Importing the necessary libraries

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB

In [35]:
df = pd.read_csv(r"C:\Users\akw97\PycharmProjects\Hateful_Speech_Detection\Datasets\HateSpeechDatasetBalanced.csv")
df.head()

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726119 entries, 0 to 726118
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Content  726119 non-null  object
 1   Label    726119 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 11.1+ MB


In [37]:
#Checking for null values in the dataset
df.isnull().sum()

Content    0
Label      0
dtype: int64

In [ ]:
#Feature Engineering for NaiveBayes Algorithm
df['Content'] = df['Content'].str.strip()
df['Content'] = df['Content'].str.replace(r'[^a-zA-Z\s]', '', regex=True)
df['Content'] = df['Content'].str.replace(r' \s+', ' ', regex=True)
df['Content'] = df['Content'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [39]:
#Using TF-IDF Vectorizer to convert text data into numerical format
tfidf = TfidfVectorizer(lowercase=True, stop_words='english', max_features=5000, ngram_range=(1, 2),max_df=0.8, min_df= 5)

X_tfidf = tfidf.fit_transform(df['Content'])

In [42]:
#Splitting the dataset into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['Label'], test_size=0.2, random_state=42)

#Creating the Multinomial Naive Bayes model
model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
model.fit(X_train, y_train)

#Making predictions on the test set
y_pred = model.predict(X_test)

#Calculating the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")



Accuracy: 78.15%


In [43]:
model = MultinomialNB()

model.fit(X_tfidf, df['Label'])

preds = model.predict(X_tfidf)
print("Accuracy: ", accuracy_score(df['Label'], preds))

Accuracy:  0.7817327462853885


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('nb', MultinomialNB())
])

# Define the grid
param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__max_df': [0.75, 0.9, 1.0],
    'tfidf__min_df': [1, 5],
    'tfidf__max_features': [None, 5000],
    'nb__alpha': [0.01, 0.1, 1, 10],
    'nb__fit_prior': [True, False]
}

# Grid search with 5-fold cross-validation
grid = GridSearchCV(pipeline, param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)

grid.fit(df['Content'], df['Label'])

print("Best Score:", grid.best_score_)
print("Best Params:", grid.best_params_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


PicklingError: Could not pickle the task to send it to the workers.